In [ ]:

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
import boto3
import json
def get_secret():

    secret_name = "chave_db_bancos"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    # Decrypts secret using the associated KMS key.
    return get_secret_value_response['SecretString']
secret = get_secret()
secret = json.loads(secret)
bucketNameDelivery = "ingestao-bigdata-delivery/"
df_fato = spark.read.parquet('s3://'+bucketNameDelivery+"fato/")
jdbc_url = "jdbc:sqlserver://" + secret.get("host") + ":" + str(secret.get("port"))
tb = "bancos.dbo.fato_bancos"
connection_mssql_options = {
    "url": jdbc_url,
    "user": secret.get("username"),
    "password": secret.get("password")}

df_fato.write.mode("overwrite").jdbc(url=jdbc_url, table=tb,properties=connection_mssql_options)
job.commit()